In [1]:
import os
import re
import pandas as pd
import nltk
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.tag.stanford import StanfordPOSTagger
from nltk.corpus import stopwords
from nltk.stem.snowball import FrenchStemmer
from nltk import NaiveBayesClassifier
from sklearn.model_selection import train_test_split

## Importing and cleaning train 

In [2]:
train= pd.read_csv('./nlp/tweets.csv', sep='^([^,]+),', engine='python', error_bad_lines=False, encoding='utf-8', index_col=[0])

In [3]:
train.head()

,polarity,statutnull
NaN,0,"- Awww, c'est un bummer. Tu devrais avoir davi..."
NaN,0,Est contrarié qu'il ne puisse pas mettre à jou...
NaN,0,J'ai plongé plusieurs fois pour la balle. A ré...
NaN,0,Tout mon corps a des démangeaisons et comme si...
NaN,0,"Non, il ne se comporte pas du tout. je suis en..."


In [4]:
#train = train.reset_index(drop=True)

In [5]:
#train.head()

In [6]:
#train.columns

In [7]:
#train['polarity'] = train['polarity'].str.replace("0","negatif").str.replace("4","positif")

In [8]:
#index_zero = train[(train['polarity'] != 'positif') & (train['polarity']!= 'negatif')].index

In [9]:
#train.drop(index_zero, inplace=True)

In [10]:
#train = train.dropna()

In [11]:
#train.to_csv('tweets_clean.csv')

In [12]:
train = train.sample(n=10000)

In [13]:

java_path = "C:/Program Files/Java/jre1.8.0_231/bin/java.exe"
os.environ['JAVAHOME'] = java_path

def clean_up(s):
    s= re.sub(r'http\S+|(www\.[^\s]+)|(@\S+)',' ',s)
    s=re.sub('[\d]',' ',s)
    s= re.sub('[^\w\s]',' ',s)
    s=s.lower().strip()
    s=word_tokenize(s)
    root_path="./stanford-postagger-full-2018-10-16/"
    pos_tagger = StanfordPOSTagger(root_path + "models/french.tagger", root_path + "stanford-postagger.jar",encoding='utf8') #instance de la classe StanfordPOSTagger en UTF-8
    z=[]
    for word, tags in pos_tagger.tag(s): 
        if tags in ('Adj', 'N', 'NC', 'V', 'Adv'):
            z.append(word)
    stop_words = stopwords.words('french')
    stop_words.append('rt')
    z=[x for x in z if x not in stop_words]
    stemmer = FrenchStemmer()
    y = list(map(lambda x: stemmer.stem(x), z))
    return y

In [14]:
train.statutnull = train.statutnull.astype(str)

In [15]:
train.statutnull = train.statutnull.apply(lambda s: clean_up(s))

KeyboardInterrupt: 

In [ ]:
train.head()

In [ ]:
train.rename(columns={'statutnull':'tweet'},inplace=True)

In [ ]:
all_text= ' '.join([x for text in train.tweet for x in text])
wordcloud = WordCloud(width=800, height=500, random_state=21,max_font_size=110).generate(all_text)
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')

In [ ]:
train.shape

In [ ]:
#train['tweet']=' '.join(train['tweet'])
train.reset_index(drop=True).head()

In [ ]:
set(train['polarity'])

## Building model 

In [ ]:
all_words = []
NUM_FEATURES = 5000
for index, value in train.tweet.iteritems():
    if value not in all_words:
        all_words += value
top_features = [x[0] for x in nltk.FreqDist(all_words).most_common(NUM_FEATURES)]

In [ ]:
df=pd.DataFrame.from_dict(nltk.FreqDist(all_words),orient='index')
df.sort_values(by=0, ascending=False).reset_index().head(10)

In [ ]:
def build_features(words):
    features = {}
    for w in top_features:
        features[w] = (w in words)
    return features

In [ ]:
featuresets = []

for index, row in train.iterrows():
    featuresets.append((build_features(row['tweet']), row['polarity']))

In [ ]:
featuresets[:2]

In [ ]:
train_set, test = train_test_split(featuresets, test_size=0.2)
classifier = NaiveBayesClassifier.train(train_set)

In [ ]:
classifier.show_most_informative_features(n=25)

In [ ]:
nltk.classify.accuracy(classifier, test)

In [ ]:
phrase='Je déteste la grève !'
phrase_clean=clean_up(phrase)
classifier.classify(dict([token,True] for token in phrase_clean))